In [21]:
import os
import time
import json
import boto3
import numpy as np
import pandas as pd
from sagemaker import get_execution_role
import sagemaker
from sagemaker.tensorflow import TensorFlow
from datetime import datetime

In [22]:
bucket = 'techsummit2023mlops'

In [23]:
sess = boto3.Session()
sm = sess.client("sagemaker")
role = get_execution_role()
sagemaker_session = sagemaker.Session(boto_session=sess)
region = boto3.Session().region_name
model_package_group_name = "tech-sum-2023-package"  # Model name in model registry
prefix = "fashion-model"
pipeline_name = "tech-sum-2023-pipeline-modular"  # SageMaker Pipeline name
current_time = time.strftime("%m-%d-%H-%M-%S", time.localtime())

In [24]:
import uuid
  
id = uuid.uuid4()

id = str(id)

print(id)

4b627783-06a9-4c73-a9e7-f4761c9daa9e


In [25]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)


In [26]:
from sagemaker.tensorflow import TensorFlowProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role
from sagemaker.workflow.pipeline_context import PipelineSession

pipeline_session = PipelineSession() # this step is making the below processor to not run now and wait for pipeline start.

region = boto3.session.Session().region_name

role = get_execution_role()

BUCKET = 'techsummit2023mlops'
train_path =  str(datetime.date(datetime.now())) + '/' + id + '/DataProcessed' + '/train'
val_path =  str(datetime.date(datetime.now())) + '/' + id + '/DataProcessed' + '/val'
test_path = str(datetime.date(datetime.now())) + '/' + id + '/DataProcessed' + '/test'

#Initialize the TensorFlowProcessor

tp = TensorFlowProcessor(
    framework_version='2.11.0',
        ----------------------------------------------------------- To be Filled ---------------------------------------------------------

    sagemaker_session=pipeline_session  # Note the last line in the above definition  "sagemaker_session=pipeline_session " has been added to make the processor part of the pipeline session
)


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


In [27]:
source_data = "s3://techsummit2023mlops/raw_data/"

In [28]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep


processor_args = tp.run(
    code='preprocessing.py',
    source_dir='script_processing',
    inputs=[
                ----------------------------------------------------------- To be Filled ---------------------------------------------------------
    ],
    outputs=[
        ----------------------------------------------------------- To be Filled ---------------------------------------------------------

        ProcessingOutput(output_name="test_data_eval", source="/opt/ml/processing/test", destination=f's3://{BUCKET}/{test_path}')    
    ]
)



In [29]:
step_process = ProcessingStep(name="PreprocessingStep", step_args=processor_args)

In [30]:
import os

import sagemaker
from sagemaker.tensorflow import TensorFlow


In [31]:
from sagemaker.inputs import TrainingInput


inputs={
        "train_output_data": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs["train_output_data"].S3Output.S3Uri,
            content_type="text/csv",
        ),
        ----------------------------------------------------------- To be Filled ---------------------------------------------------------

    }


In [32]:
bucket_name = "XXXXX"
model_dir = "XXXXXXXXX"
checkpoint_in_bucket="checkpoints"

# this path is for the upload location of keras weights for training job
model_trained_base_loc = str(datetime.date(datetime.now())) 
model_dir= ----------------------------------------------------------- To be Filled ---------------------------------------------------------


# The S3 URI to store the checkpoints
checkpoint_s3_bucket="s3://{}/{}/{}/{}".format(bucket_name, model_trained_base_loc, id, checkpoint_in_bucket)

# The local path where the model will save its checkpoints in the training container
checkpoint_local_path="/opt/ml/checkpoints"

In [33]:
hyperparameters={
    "output_dir": "/opt/ml/model",
    "bucket_name": bucket_name,
    "model_dir": model_dir
    }

In [34]:
from sagemaker.tensorflow import TensorFlow
from sagemaker.workflow.steps import TrainingStep
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.pipeline import Pipeline


import time

train_instance_type = "ml.m5.xlarge"

estimator = TensorFlow(
        ----------------------------------------------------------- To be Filled ---------------------------------------------------------

    base_job_name="TrainingStep",
    checkpoint_s3_uri=checkpoint_s3_bucket,       ## https://docs.aws.amazon.com/sagemaker/latest/dg/model-checkpoints.html
        ----------------------------------------------------------- To be Filled ---------------------------------------------------------

)

# Use the tf2_estimator in a Sagemaker pipelines ProcessingStep.
# NOTE how the input to the training job directly references the output of the previous step.
step_train_model = TrainingStep(
    name="Training-Step-Model",
    estimator=estimator,
    inputs=inputs
)

In [35]:
from sagemaker.tensorflow import TensorFlowProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role
from sagemaker.workflow.pipeline_context import PipelineSession


region = boto3.session.Session().region_name

pipeline_session = PipelineSession() # this step is making the below processor to not run now and wait for pipeline start.


role = get_execution_role()

eval_path = str(datetime.date(datetime.now())) + '/' + id + '/DataEvaluation' + '/metrics/'

# eval_path = 'DataEvaluation/metrics/'

#Initialize the TensorFlowProcessor


tp = TensorFlowProcessor(
    framework_version='2.11.0',
    role=get_execution_role(),
    instance_type='ml.m5.xlarge',
    instance_count=1,
    base_job_name='frameworkprocessor-TF',
    py_version='py39',
    sagemaker_session=pipeline_session,
)

#Run the processing job
evaluation_args = tp.run(
    code='evaluation.py',
    source_dir='script_evaluation',
    inputs=[
        ----------------------------------------------------------- To be Filled ---------------------------------------------------------
        ProcessingInput(input_name='test_data_dummy',source=step_train_model.properties.ModelArtifacts.S3ModelArtifacts, destination="/opt/ml/processing/test_data_dummy")
    ],
    outputs=[
        ProcessingOutput(output_name="eval_output_data", ---------Fillin-------, destination=f's3://{BUCKET}/{eval_path}'),
    ]
)




INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


In [36]:
step_evaluate = ProcessingStep(name="EvaluationStep", step_args=evaluation_args)

In [37]:
from sagemaker.model import Model

image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version="1.11.0",
    py_version="py39",
    instance_type="ml.m5.xlarge",
    image_scope="inference",
)

model = Model(
    image_uri=image_uri,
    model_data=step_train_model.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=pipeline_session,
    role=role,
)

# Define the ModelStep by providing the return values from model.create() as the step arguments.

from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.model_step import ModelStep

step_create_model = ModelStep(
    name="tech-sum-2023-CreateModel",
    step_args=model.create(instance_type="ml.m5.large", accelerator_type="ml.eia1.medium"),
)



INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py39.


In [38]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation_metrics.json".format(
            step_evaluate.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json",
    )
)

customer_metadata_properties = {"id": id}


register_args = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    model_metrics=model_metrics,
    customer_metadata_properties=customer_metadata_properties
    
)
step_register = ModelStep(name="tech-sum-2023-RegisterModel", step_args=register_args)



INFO:sagemaker.processing:Uploaded script_evaluation to s3://sagemaker-ap-southeast-2-970770904251/frameworkprocessor-TF-2023-09-03-15-31-42-708/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ap-southeast-2-970770904251/frameworkprocessor-TF-2023-09-03-15-31-42-708/source/runproc.sh


Using provided s3_resource


In [39]:
from sagemaker.workflow.pipeline import Pipeline
pipeline = Pipeline(
    name=pipeline_name,
    steps=[  ----------------------------------------------------------- To be Filled ---------------------------------------------------------]
)

pipeline.upsert(role_arn=role)
definition = json.loads(pipeline.definition())
definition
execution = pipeline.start()
execution.wait()

INFO:sagemaker.processing:Uploaded script_processing to s3://sagemaker-ap-southeast-2-970770904251/tech-sum-2023-pipeline-modular/code/84cbb526e57fc4f29b718ecc2ee37e52/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ap-southeast-2-970770904251/tech-sum-2023-pipeline-modular/code/54f0ef6bee583ff9186b762aaf572190/runproc.sh


Using provided s3_resource
Using provided s3_resource


INFO:sagemaker.estimator:SMDebug Does Not Currently Support                         Distributed Training Jobs With Checkpointing Enabled
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.processing:Uploaded script_evaluation to s3://sagemaker-ap-southeast-2-970770904251/tech-sum-2023-pipeline-modular/code/cb57a00e61957675932335e495abbebb/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ap-southeast-2-970770904251/tech-sum-2023-pipeline-modular/code/2c207c809cb0e0e9a1d77e5247f961f9/runproc.sh


Using provided s3_resource


INFO:sagemaker.processing:Uploaded script_processing to s3://sagemaker-ap-southeast-2-970770904251/tech-sum-2023-pipeline-modular/code/84cbb526e57fc4f29b718ecc2ee37e52/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ap-southeast-2-970770904251/tech-sum-2023-pipeline-modular/code/54f0ef6bee583ff9186b762aaf572190/runproc.sh
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource
Using provided s3_resource
Using provided s3_resource


INFO:sagemaker.processing:Uploaded script_evaluation to s3://sagemaker-ap-southeast-2-970770904251/tech-sum-2023-pipeline-modular/code/cb57a00e61957675932335e495abbebb/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ap-southeast-2-970770904251/tech-sum-2023-pipeline-modular/code/2c207c809cb0e0e9a1d77e5247f961f9/runproc.sh
INFO:sagemaker.processing:Uploaded script_processing to s3://sagemaker-ap-southeast-2-970770904251/tech-sum-2023-pipeline-modular/code/84cbb526e57fc4f29b718ecc2ee37e52/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ap-southeast-2-970770904251/tech-sum-2023-pipeline-modular/code/54f0ef6bee583ff9186b762aaf572190/runproc.sh
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource
Using provided s3_resource


INFO:sagemaker.processing:Uploaded script_evaluation to s3://sagemaker-ap-southeast-2-970770904251/tech-sum-2023-pipeline-modular/code/cb57a00e61957675932335e495abbebb/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ap-southeast-2-970770904251/tech-sum-2023-pipeline-modular/code/2c207c809cb0e0e9a1d77e5247f961f9/runproc.sh


Using provided s3_resource
